In [30]:
// id : ex_5.1_5.3

sealed trait Stream[+A] {
    
    def headOption: Option[A] = this match {
        case Empty => None
        case Cons(h, t) => Some(h())
    }
    
    def toList: List[A] = this match {
        case Empty => Nil
        case Cons(h, t) => h() :: t().toList
    }
    
    def foldRight[B](z: => B)(f: (A, => B) => B): B = this match {
        case Cons(h, t) => f(h(), t().foldRight(z)(f))
        case _ => z
    }
    
    def exists(p: A => Boolean): Boolean =
        foldRight(false)((a, b) => p(a) || b)
    
    // ex 5.1
    def take(n : Int) : Stream[A] = {
        if( n <= 0) {
            Empty
        } else {
            this match {
                case Empty => Empty
                case Cons(h, t) => Cons(h, () => t().take(n-1))
            }  
        }
    }
    
    // ex 5.2
    def drop(n : Int) : Stream[A] = {
        if( n <= 0) {
            this
        } else {
            this match {
                case Empty => Empty
                case Cons(h, t) => t().drop(n-1)
            }  
        }
    }
    
    // ex 5.3
    def takeWhile(p: A => Boolean) : Stream[A] = {
        this match {
            case Empty => Empty
            case Cons(h, t) => if (p(h())) {
                Cons(h, () => t().takeWhile(p))
            } else {
                Empty
            }
        }
    }
    
    // ex 5.4
    def forAll(p: A => Boolean): Boolean = {
        this match {
            case Empty => true
            case Cons(h, t) => if (!p(h())) {
                false
            } else {
                t().forAll(p)
            }
        }
    }
    
    // ex 5.5 Implement takeWhile with FoldRight
    // foldRight[B](z: => B)(f: (A, => B) => B): B
    // f(h(), t().foldRight(z)(f))
    def takeWhileFR(p: A => Boolean) : Stream[A] = 
        foldRight(Empty.asInstanceOf[Stream[A]])((h, t) => if (p(h)) { 
            Cons(() => h, () => t) 
        } else {
            Empty
        })
    
    // ex 5.6 Implement headOption using FoldRight
    def headOptionFR: Option[A] = this.foldRight(None.asInstanceOf[Option[A]])((h, _) => Some(h))
    
    // ex 5.7 Implment map, filter append and flatMap using foldRight.
    // append should be non-strict in its argument.
    def map[B](f: A => B): Stream[B] = 
        this.foldRight(Empty.asInstanceOf[Stream[B]])((h, t) => Cons(() => f(h), () => t))
    
    def filter[B](p: A => Boolean): Stream[A] =
        this.foldRight(Empty.asInstanceOf[Stream[A]])((h, t) => {
            if (p(h)) {
                Cons(() => h, () => t)
            } else {
                t
            }
        })
    
    def append[B >: A](a2 : Stream[B]) : Stream[B] = 
        this.foldRight(a2)((h, t) => {
            Cons(() => h, () => t)
        }) 
}

case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A]

object Stream {
    def cons[A](hd: => A, tl: => Stream[A]): Stream [A] = {
        lazy val head = hd
        lazy val tail = tl
        Cons(() => head, () => tail)
    }
    
    def empty[A]: Stream[A] = Empty
    
    def apply[A](as: A*): Stream[A] = 
        if (as.isEmpty) empty else cons(as.head, apply(as.tail: _*))
}

$line42.$read$$iw$$iw$Stream$@1bcccba9

In [18]:
// id : try ex_5.1_5.3
println("headOption", Stream().headOption)
println("headOption", Stream(1,2,3).headOption)
println("take(2)",  Stream(1,2,3,4,5,6).take(2).toList)
println("drop(4)",  Stream(1,2,3,4,5,6).drop(4).toList)
println("takeWhile(x < 5)",  Stream(1,2,3,4,5,6).takeWhile((x : Int) => x < 5).toList)
println("forAll(x>0)", Stream(1,2,3,4,5,6).forAll((x : Int) => x > 0))
println("forAll(x>0)", Stream(1,2,3,4,5,6,-1).forAll((x : Int) => x > 0))

(headOption,None)
(headOption,Some(1))
(take(2),List(1, 2))
(drop(4),List(5, 6))
(takeWhile(x < 5),List(1, 2, 3, 4))
(forAll(x>0),true)
(forAll(x>0),false)


null

In [19]:
// id : try ex_5.5
println("takeWhileFR(x < 5)",  Stream(1,2,3,4,5,6).takeWhileFR((x : Int) => x < 5).toList)
println("takeWhileFR(x < 5)",  Stream().takeWhileFR((x : Int) => x < 5).toList)
println("takeWhileFR(x < 5)",  Stream(4).takeWhileFR((x : Int) => x < 5).toList)
println("takeWhileFR(x < 5)",  Stream(6).takeWhileFR((x : Int) => x < 5).toList)

(takeWhileFR(x < 5),List(1, 2, 3, 4))
(takeWhileFR(x < 5),List())
(takeWhileFR(x < 5),List(4))
(takeWhileFR(x < 5),List())


null

In [21]:
// id : try ex_5.6
println("headOptionFR", Stream().headOptionFR)
println("headOptionFR", Stream(1,2,3).headOptionFR)

(headOptionFR,None)
(headOptionFR,Some(1))


null

In [31]:
// id : try ex_5.7
println("map", Stream(1,2,3,4,5).map(x => "<" + x.toString + ">").toList)
println("filter", Stream(1,2,3,4,5).filter(x => x > 3).toList)
println("append", Stream(1,2).append(Stream(3,4)).toList)

(map,List(<1>, <2>, <3>, <4>, <5>))
(filter,List(4, 5))
(append,List(1, 2, 3, 4))


null